In [ ]:
from multiprocessing import Process, Pipe
from os import getpid
from datetime import datetime

In [ ]:
def local_time(counter):
  return ' (LAMPORT_TIME={}, LOCAL_TIME={})'.format(counter,datetime.now())

    

In [ ]:
def calc_recv_timestamp(recv_time_stamp, counter):
  return max(recv_time_stamp, counter) + 1
    

In [ ]:
def event(pid, counter):
    counter += 1
    print('Something happened in {} !'.\
          format(pid) + local_time(counter))
    return counter

In [ ]:
def send_message(pipe, pid, counter):
    counter += 1
    pipe.send(('Empty shell', counter))
    print('Message sent from ' + str(pid) + local_time(counter))
    return counter

In [ ]:
def recv_message(pipe, pid, counter):
    message, timestamp = pipe.recv()
    counter = calc_recv_timestamp(timestamp, counter)
    print('Message received at ' + str(pid)  + local_time(counter))
    return counter

In [ ]:
def process_one(pipe12):
    pid = getpid()
    counter = 0
    counter = event(pid, counter)
    counter = send_message(pipe12, pid, counter)
    counter  = event(pid, counter)
    counter = recv_message(pipe12, pid, counter)
    counter  = event(pid, counter)

def process_two(pipe21, pipe23):
    pid = getpid()
    counter = 0
    counter = recv_message(pipe21, pid, counter)
    counter = send_message(pipe21, pid, counter)
    counter = send_message(pipe23, pid, counter)
    counter = recv_message(pipe23, pid, counter)


def process_three(pipe32):
    pid = getpid()
    counter = 0
    counter = recv_message(pipe32, pid, counter)
    counter = send_message(pipe32, pid, counter)

In [ ]:
if __name__ == '__main__':
    oneandtwo, twoandone = Pipe()
    twoandthree, threeandtwo = Pipe()

    process1 = Process(target=process_one, 
                       args=(oneandtwo,))
    process2 = Process(target=process_two, 
                       args=(twoandone, twoandthree))
    process3 = Process(target=process_three, 
                       args=(threeandtwo,))

    process1.start()
    process2.start()
    process3.start()

    process1.join()
    process2.join()
    process3.join()

Something happened in 910 ! (LAMPORT_TIME=1, LOCAL_TIME=2023-01-31 10:58:01.297914)
Message received at 913 (LAMPORT_TIME=3, LOCAL_TIME=2023-01-31 10:58:01.309741)Message sent from 910 (LAMPORT_TIME=2, LOCAL_TIME=2023-01-31 10:58:01.308846)

Message sent from 913 (LAMPORT_TIME=4, LOCAL_TIME=2023-01-31 10:58:01.333026)Something happened in 910 ! (LAMPORT_TIME=3, LOCAL_TIME=2023-01-31 10:58:01.328395)
Message received at 918 (LAMPORT_TIME=6, LOCAL_TIME=2023-01-31 10:58:01.338362)Message sent from 913 (LAMPORT_TIME=5, LOCAL_TIME=2023-01-31 10:58:01.338004)

Message received at 910 (LAMPORT_TIME=5, LOCAL_TIME=2023-01-31 10:58:01.347137)

Something happened in 910 ! (LAMPORT_TIME=6, LOCAL_TIME=2023-01-31 10:58:01.353812)
Message sent from 918 (LAMPORT_TIME=7, LOCAL_TIME=2023-01-31 10:58:01.365949)Message received at 913 (LAMPORT_TIME=8, LOCAL_TIME=2023-01-31 10:58:01.368845)

